<a href="https://colab.research.google.com/github/bea-codes/otimizacao-hiperparametros/blob/main/Otimiza%C3%A7%C3%A3o_de_hiperpar%C3%A2metros_de_redes_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade Otimização de Hiperparâmetros
### Disciplina: Inteligência Artificial Aplicada
---
Dupla: Beatriz Barbosa e Davi Galdivo

# Código

## Setup

In [144]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Modelo

### New datase

In [145]:
dataset = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()

In [146]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [148]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


### Hiperparâmetros

In [221]:
# Número de camadas escondidas -> [1,5,7,10]
num_hidden_layers = "1" # @param [1,5,7,10]
num_hidden_layers = int(num_hidden_layers)
# Número de neurônios por camada escondida -> [25, 50, 100, 200, 500]
num_unit_per_layer = "500" # @param [25,50,100,200,500]
num_unit_per_layer = int(num_unit_per_layer)

# Epochs
num_epochs = 10

# Batch size
batch_size = 5

### Criação

In [208]:
# Função cria modelo de acordo com os argumentos
def create_model(num_hidden_layers, num_unit_per_layer):
  model = keras.Sequential(name="Modelo")
  # model.add(keras.Input(shape=(16,))) # Não vai aparecer no model.summary().
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  for n in range(num_hidden_layers):
    model.add(keras.layers.Dense(units=num_unit_per_layer, activation='relu'))

  model.add(keras.layers.Dense(10, activation='softmax'))

  return model

In [222]:
model = create_model(num_hidden_layers, num_unit_per_layer)
model.summary()

Model: "Modelo"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_28 (Flatten)        (None, 784)               0         
                                                                 
 dense_261 (Dense)           (None, 500)               392500    
                                                                 
 dense_262 (Dense)           (None, 10)                5010      
                                                                 
Total params: 397510 (1.52 MB)
Trainable params: 397510 (1.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Compilando o modelo

In [223]:
model = create_model(num_hidden_layers, num_unit_per_layer)
model.summary()

Model: "Modelo"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_29 (Flatten)        (None, 784)               0         
                                                                 
 dense_263 (Dense)           (None, 500)               392500    
                                                                 
 dense_264 (Dense)           (None, 10)                5010      
                                                                 
Total params: 397510 (1.52 MB)
Trainable params: 397510 (1.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [224]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


### Treinando o modelo

In [225]:
earlystopping = keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)

In [ ]:
model.fit(train_images, train_labels, epochs=num_epochs, callbacks=[earlystopping])

### Predições

In [227]:
predictions = model.predict(test_images)

313/313 [==============================] - 1s 2ms/step


In [228]:
print(test_images.shape)
print(predictions.shape)


(10000, 28, 28)
(10000, 10)


In [229]:
def exibir_predicoes(predictions, num):
  for p in range(num):
    predicao = np.argmax(predictions[p])
    classe_real = test_labels[p]
    print(f"Predição do modelo: {class_names[predicao]}\t|\t Classe real: {class_names[classe_real]}")

In [230]:
exibir_predicoes(predictions, 15)

Predição do modelo: Ankle boot	|	 Classe real: Ankle boot
Predição do modelo: Pullover	|	 Classe real: Pullover
Predição do modelo: Trouser	|	 Classe real: Trouser
Predição do modelo: Trouser	|	 Classe real: Trouser
Predição do modelo: Shirt	|	 Classe real: Shirt
Predição do modelo: Trouser	|	 Classe real: Trouser
Predição do modelo: Coat	|	 Classe real: Coat
Predição do modelo: Shirt	|	 Classe real: Shirt
Predição do modelo: Sandal	|	 Classe real: Sandal
Predição do modelo: Sneaker	|	 Classe real: Sneaker
Predição do modelo: Coat	|	 Classe real: Coat
Predição do modelo: Sandal	|	 Classe real: Sandal
Predição do modelo: Sandal	|	 Classe real: Sneaker
Predição do modelo: Dress	|	 Classe real: Dress
Predição do modelo: Coat	|	 Classe real: Coat


# Tests


## Download da base de dados

In [ ]:
# Limpando o ambiente
!rm -rf sample_data

In [ ]:
!wget https://archive.ics.uci.edu/static/public/111/zoo.zip

--2023-10-29 14:21:01--  https://archive.ics.uci.edu/static/public/111/zoo.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘zoo.zip’

zoo.zip                 [ <=>                ]   2.44K  --.-KB/s    in 0s      

2023-10-29 14:21:01 (999 MB/s) - ‘zoo.zip’ saved [2496]



In [ ]:
!unzip zoo.zip -d database

Archive:  zoo.zip
  inflating: database/Index          
  inflating: database/zoo.data       
  inflating: database/zoo.names      


In [ ]:
# Base original do UCI não tem o nome das colunas. Pegamos essa base já modificada
# Com os nomes corretos de um repositório do Github pra facilitar o trabalho.

!wget https://github.com/sharan91/zoo_classification/raw/master/zoo.csv

--2023-10-29 14:21:01--  https://github.com/sharan91/zoo_classification/raw/master/zoo.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharan91/zoo_classification/master/zoo.csv [following]
--2023-10-29 14:21:02--  https://raw.githubusercontent.com/sharan91/zoo_classification/master/zoo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4368 (4.3K) [text/plain]
Saving to: ‘zoo.csv’

zoo.csv             100%[===================>]   4.27K  --.-KB/s    in 0s      

2023-10-29 14:21:02 (52.8 MB/s) - ‘zoo.csv’ saved [4368/4368]



### Old database

In [ ]:
df = pd.read_csv('./zoo.csv')

In [ ]:
df.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [ ]:
df['class_type'].value_counts()

1    41
2    20
4    13
7    10
6     8
3     5
5     4
Name: class_type, dtype: int64

In [ ]:
y = df['class_type']
X = df.drop(['animal_name','class_type'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
with open('./database/zoo.names') as f:
  print(f.read())

1. Title: Zoo database

2. Source Information
   -- Creator: Richard Forsyth
   -- Donor: Richard S. Forsyth 
             8 Grosvenor Avenue
             Mapperley Park
             Nottingham NG3 5DX
             0602-621676
   -- Date: 5/15/1990
 
3. Past Usage:
   -- None known other than what is shown in Forsyth's PC/BEAGLE User's Guide.

4. Relevant Information:
   -- A simple database containing 17 Boolean-valued attributes.  The "type"
      attribute appears to be the class attribute.  Here is a breakdown of
      which animals are in which type: (I find it unusual that there are
      2 instances of "frog" and one of "girl"!)

      Class# Set of animals:
      ====== ===============================================================
           1 (41) aardvark, antelope, bear, boar, buffalo, calf,
                  cavy, cheetah, deer, dolphin, elephant,
                  fruitbat, giraffe, girl, goat, gorilla, hamster,
                  hare, leopard, lion, lynx, mink, mole, mo